In [2]:
import torch
import torch.nn as nn
import numpy as np
import random

# --- Same Preprocessing ---

corpus = "deep learning is fun and deep learning is powerful"
words = corpus.split()
vocab = sorted(set(words))
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

training_sentences = [
    ['deep', 'learning', 'is', 'fun'],
    ['learning', 'is', 'powerful'],
    ['deep', 'learning', 'is', 'powerful'],
    ['learning', 'is', 'fun', 'and', 'powerful']
]

seq_length = 3
input_seqs = []
target_words = []

for sentence in training_sentences:
    if len(sentence) <= seq_length:
        continue
    for i in range(len(sentence) - seq_length):
        input_seq = sentence[i: i + seq_length]
        target_word = sentence[i + seq_length]
        input_tensor = torch.tensor([word_to_idx[word] for word in input_seq], dtype=torch.long)
        target_tensor = torch.tensor(word_to_idx[target_word], dtype=torch.long)
        input_seqs.append(input_tensor)
        target_words.append(target_tensor)

# --- Model Definition ---
class WordRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(WordRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        out, hidden = self.rnn(embedded, hidden)
        out = self.fc(out[:, -1, :])  # Last time step
        return out, hidden

# --- Training ---
embedding_dim = 10
hidden_dim = 16
model = WordRNN(len(vocab), embedding_dim, hidden_dim)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 300
for epoch in range(1, num_epochs + 1):
    total_loss = 0
    model.train()
    for input_tensor, target_tensor in zip(input_seqs, target_words):
        hidden = torch.zeros(1, 1, hidden_dim)
        input_tensor = input_tensor.unsqueeze(0)
        optimizer.zero_grad()
        output, hidden = model(input_tensor, hidden)
        loss = loss_fn(output, target_tensor.unsqueeze(0))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 50 == 0:
        print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item():.4f}")

# --- Softmax Sampling Function ---
def sample_from_probs(probs, temperature=1.0):
    probs = torch.softmax(probs / temperature, dim=-1).detach().cpu().numpy()
    return np.random.choice(len(probs), p=probs)

# --- Generation Function ---
def generate_text(model, seed_words, length=5, temperature=1.0):
    model.eval()
    generated = seed_words[:]
    input_seq = [word_to_idx[word] for word in seed_words]

    hidden = torch.zeros(1, 1, hidden_dim)

    for _ in range(length):
        input_tensor = torch.tensor(input_seq).unsqueeze(0)
        output, hidden = model(input_tensor, hidden)
        next_idx = sample_from_probs(output[0], temperature)
        next_word = idx_to_word[next_idx]
        generated.append(next_word)
        input_seq = input_seq[1:] + [next_idx]  # Slide window

    return ' '.join(generated)

# --- Try Generating Sentences ---
seed = ['deep', 'learning', 'is']
print("\nGenerated Text (temp=1.0):")
print(generate_text(model, seed_words=seed, length=6, temperature=1.0))

print("\nGenerated Text (temp=0.5):")
print(generate_text(model, seed_words=seed, length=6, temperature=0.5))

print("\nGenerated Text (temp=1.5):")
print(generate_text(model, seed_words=seed, length=6, temperature=1.5))


Epoch 50/300, Loss: 0.0297
Epoch 100/300, Loss: 0.0144
Epoch 150/300, Loss: 0.0094
Epoch 200/300, Loss: 0.0070
Epoch 250/300, Loss: 0.0055
Epoch 300/300, Loss: 0.0046

Generated Text (temp=1.0):
deep learning is powerful and powerful deep powerful and

Generated Text (temp=0.5):
deep learning is powerful and powerful and powerful and

Generated Text (temp=1.5):
deep learning is powerful and powerful powerful and powerful
